# Ollama + OpenAI + Python

## ১. মডেলের নাম নির্ধারণ করুন

যদি আপনি "phi3:mini" ছাড়া অন্য কোনো মডেল ব্যবহার করেন, তাহলে নিচের সেলে মান পরিবর্তন করুন। এই ভেরিয়েবলটি নোটবুকের কোডে সর্বত্র ব্যবহার করা হবে।


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. ওপেন এআই ক্লায়েন্ট সেটআপ করুন

সাধারণত ওপেন এআই ক্লায়েন্ট OpenAI.com বা Azure OpenAI এর সাথে বড় ভাষার মডেলের সাথে যোগাযোগ করার জন্য ব্যবহৃত হয়।  
তবে, এটি Ollama এর সাথেও ব্যবহার করা যেতে পারে, কারণ Ollama "http://localhost:11434/v1" এ একটি OpenAI-সামঞ্জস্যপূর্ণ এন্ডপয়েন্ট প্রদান করে।


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## ৩. একটি চ্যাট সম্পন্ন তৈরি করুন

এখন আমরা OpenAI SDK ব্যবহার করে একটি কথোপকথনের জন্য উত্তর তৈরি করতে পারি। এই অনুরোধটি বিড়ালদের নিয়ে একটি হাইকু তৈরি করবে:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## ৪. প্রম্পট ইঞ্জিনিয়ারিং

ভাষার মডেলে পাঠানো প্রথম বার্তাটি "সিস্টেম মেসেজ" বা "সিস্টেম প্রম্পট" নামে পরিচিত, যা মডেলের জন্য সামগ্রিক নির্দেশনা নির্ধারণ করে।  
আপনি আপনার নিজস্ব কাস্টম সিস্টেম প্রম্পট প্রদান করতে পারেন, যা ভাষার মডেলকে ভিন্নভাবে আউটপুট তৈরি করতে নির্দেশনা দেয়।  
নিচের `SYSTEM_MESSAGE` পরিবর্তন করুন যাতে এটি আপনার প্রিয় বিখ্যাত সিনেমা/টিভি চরিত্রের মতো উত্তর দেয়, অথবা অন্যান্য সিস্টেম প্রম্পটের জন্য [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) থেকে অনুপ্রেরণা নিন।  

যখন আপনি সিস্টেম মেসেজটি কাস্টমাইজ করবেন, তখন `USER_MESSAGE`-এ প্রথম ব্যবহারকারীর প্রশ্ন প্রদান করুন।  


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## ৫. কিছু উদাহরণ

একটি ভাষা মডেলকে নির্দেশনা দেওয়ার আরেকটি উপায় হলো "কিছু উদাহরণ" প্রদান করা, যেখানে প্রশ্ন/উত্তরের একটি ধারাবাহিক উদাহরণ দেখানো হয় যা নির্দেশ করে এটি কীভাবে সাড়া দেবে।

নিচের উদাহরণটি একটি ভাষা মডেলকে শিক্ষকের সহকারী হিসেবে কাজ করানোর চেষ্টা করে, যেখানে কিছু প্রশ্ন এবং উত্তরের উদাহরণ দেওয়া হয়েছে যা একজন সহকারী দিতে পারে, এবং তারপর মডেলকে এমন একটি প্রশ্নের মুখোমুখি করা হয়েছে যা একজন শিক্ষার্থী জিজ্ঞাসা করতে পারে।

প্রথমে এটি চেষ্টা করুন, তারপর `SYSTEM_MESSAGE`, `EXAMPLES`, এবং `USER_MESSAGE` পরিবর্তন করে একটি নতুন পরিস্থিতির জন্য এটি ব্যবহার করুন।


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## ৬. রিট্রিভাল অগমেন্টেড জেনারেশন

RAG (Retrieval Augmented Generation) একটি পদ্ধতি যা একটি ভাষার মডেলকে নির্দিষ্ট কোনো ডোমেইনের প্রশ্নের সঠিক উত্তর দিতে সক্ষম করে। এটি প্রথমে একটি জ্ঞান উৎস থেকে প্রাসঙ্গিক তথ্য সংগ্রহ করে এবং তারপর সেই তথ্যের ভিত্তিতে উত্তর তৈরি করে।

আমরা একটি স্থানীয় CSV ফাইল সরবরাহ করেছি, যেখানে হাইব্রিড গাড়ি সম্পর্কিত তথ্য রয়েছে। নিচের কোডটি CSV ফাইলটি পড়ে, ব্যবহারকারীর প্রশ্নের সাথে মিলে যাওয়া তথ্য খুঁজে বের করে এবং তারপর পাওয়া তথ্যের ভিত্তিতে একটি উত্তর তৈরি করে। মনে রাখবেন, এটি আগের উদাহরণগুলোর তুলনায় বেশি সময় নেবে, কারণ এটি মডেলে বেশি তথ্য পাঠায়। যদি আপনি লক্ষ্য করেন যে উত্তরটি এখনও ডেটার সাথে সঠিকভাবে সংযুক্ত নয়, তাহলে আপনি সিস্টেম ইঞ্জিনিয়ারিং চেষ্টা করতে পারেন বা অন্য মডেল ব্যবহার করতে পারেন। সাধারণত, RAG বড় মডেল বা SLM-এর ফাইন-টিউন করা সংস্করণের সাথে বেশি কার্যকর।


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
